## Import required libraries for dataset management, model building, training, and visualization.

In [ ]:
import os
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from collections import defaultdict
import random
import kagglehub
import shutil
import glob
import re
from tqdm import tqdm  # For progress tracking.

## Dataset and Utilities

In [ ]:
from google.colab import files
train_data = files.upload()
test_data = files.upload()

print(list(train_data.keys()))
print(list(test_data.keys()))